Basically our data is divided into two data sets, one is the dataset of LTFS which contains information of all the existing customers or past customers of l&t and in which period they have given top up , second data is the whole loan history of the customers, this may include loan from any non banking organisation like LTFS or LTFS itself and also from other institutes.
Our aim to classify the right bucket period in which LTFS can give loans , so we will find out the past history of the customers and add new features to our main dataset provided by the LTFS and will then do our analysis.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Data/train_bureau.csv')

In [ ]:
data.head(5)

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,...,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE
0,1,False,PRIMARY,Overdraft,NAB,2018-04-30 00:00:00,Individual,Delinquent,2015-10-05 00:00:00,NaN,...,NaN,"37,873",0.0,Standard,"20180430,20180331,",030000,"37873,12820,","37873,,",",,",NaN
1,1,False,PRIMARY,Auto Loan (Personal),NAB,2019-12-31 00:00:00,Individual,Active,2018-03-19 00:00:00,NaN,...,F03,NaN,0.0,Standard,"20191231,20191130,20191031,20190930,20190831,2...",0000000000000000000000000000000000000000000000...,"20797,21988,23174,24341,25504,26648,27780,2891...",",,,,,,,,,,,,,,,,,,,,1452,,",",,,,,,,,,,,,,,,,,,,,,,",36.0
2,1,True,PRIMARY,Tractor Loan,NBF,2020-01-31 00:00:00,Individual,Active,2019-08-30 00:00:00,NaN,...,NaN,0,0.0,NaN,"20200131,20191231,20191130,20191031,20190930,2...",000000000000000000,"116087,116087,145000,145000,145000,145000,","0,0,0,0,0,0,",",,,,,,",NaN
3,1,True,PRIMARY,Auto Loan (Personal),NBF,2017-09-30 00:00:00,Individual,Closed,2013-09-27 00:00:00,2017-09-21 00:00:00,...,NaN,0,0.0,NaN,"20170930,20170801,20170731,20170630,20170531,2...",000DDD0270260270260270240270270000320000000000...,"0,,15925,23754,31494,39147,46713,54194,61590,6...","0,,1014,1014,1014,1014,1014,1014,1014,983,0,92...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN
4,1,True,PRIMARY,Tractor Loan,NBF,2016-02-29 00:00:00,Individual,Closed,2012-02-10 00:00:00,2016-02-01 00:00:00,...,NaN,0,0.0,NaN,"20160229,20160131,20151231,20151130,20151031,2...",0000000000000000000000000000000000000000000000...,"0,0,23658,23321,22989,46321,45662,45012,68030,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN


In [ ]:
data.shape

(560844, 25)

5.6 Lakh rows and 25 columns

In [ ]:
df=data.copy()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560844 entries, 0 to 560843
Data columns (total 25 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ID                         560844 non-null  int64  
 1   SELF-INDICATOR             560844 non-null  bool   
 2   MATCH-TYPE                 560844 non-null  object 
 3   ACCT-TYPE                  560844 non-null  object 
 4   CONTRIBUTOR-TYPE           560844 non-null  object 
 5   DATE-REPORTED              557161 non-null  object 
 6   OWNERSHIP-IND              560844 non-null  object 
 7   ACCOUNT-STATUS             560844 non-null  object 
 8   DISBURSED-DT               528694 non-null  object 
 9   CLOSE-DT                   309086 non-null  object 
 10  LAST-PAYMENT-DATE          241561 non-null  object 
 11  CREDIT-LIMIT/SANC AMT      15159 non-null   object 
 12  DISBURSED-AMT/HIGH CREDIT  560844 non-null  object 
 13  INSTALLMENT-AMT            14

In [ ]:
df.duplicated().sum()

3864

Removing duplicated values

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
# AS dropping the values changes the index of the data
df.reset_index(inplace=True, drop=True)  #We can use the drop parameter to avoid the old index being added as a column:

In [ ]:
df.columns

Index(['ID', 'SELF-INDICATOR', 'MATCH-TYPE', 'ACCT-TYPE', 'CONTRIBUTOR-TYPE',
       'DATE-REPORTED', 'OWNERSHIP-IND', 'ACCOUNT-STATUS', 'DISBURSED-DT',
       'CLOSE-DT', 'LAST-PAYMENT-DATE', 'CREDIT-LIMIT/SANC AMT',
       'DISBURSED-AMT/HIGH CREDIT', 'INSTALLMENT-AMT', 'CURRENT-BAL',
       'INSTALLMENT-FREQUENCY', 'OVERDUE-AMT', 'WRITE-OFF-AMT', 'ASSET_CLASS',
       'REPORTED DATE - HIST', 'DPD - HIST', 'CUR BAL - HIST',
       'AMT OVERDUE - HIST', 'AMT PAID - HIST', 'TENURE'],
      dtype='object')

##Current Balance

In [ ]:
#OUTSTANDING AMOUNT A BORROWER STILL OWES
df1=df['CUR BAL - HIST'].str.split(',', expand=True)
df1=df1.apply(pd.to_numeric)
df1.head(5)

#Expand=True means list of substrings should be expanded into separate columns in the DataFrame
#incase of False, it will return pandas series

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,37873.0,12820.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20797.0,21988.0,23174.0,24341.0,25504.0,26648.0,27780.0,28910.0,30020.0,31128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,116087.0,116087.0,145000.0,145000.0,145000.0,145000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,NaN,15925.0,23754.0,31494.0,39147.0,46713.0,54194.0,61590.0,68903.0,...,187265.0,193157.0,198983.0,204743.0,210438.0,216069.0,221636.0,227140.0,232582.0,NaN
4,0.0,0.0,23658.0,23321.0,22989.0,46321.0,45662.0,45012.0,68030.0,67062.0,...,177832.0,175302.0,196467.0,193672.0,190917.0,211860.0,208846.0,205875.0,226605.0,NaN


In [ ]:
df1.isnull().sum()

0      56432
1     121086
2     104996
3     117520
4     133630
5     144940
6     160413
7     177264
8     181286
9     190964
10    198923
11    207248
12    223641
13    246085
14    257476
15    264892
16    274822
17    284007
18    291070
19    296261
20    304026
21    316204
22    321556
23    328163
24    336049
25    347841
26    357442
27    366304
28    375366
29    380311
30    386167
31    396006
32    402843
33    410654
34    420044
35    427482
36    556980
dtype: int64

In [ ]:
df1.drop([36],axis=1,inplace = True) # omiting last additional entry

In [ ]:
#Difference dataframe
diffbal=df1.diff(axis=1)

In [ ]:
diffbal.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,NaN,-25053.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1191.0,1186.0,1167.0,1163.0,1144.0,1132.0,1130.0,1110.0,1108.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,0.0,28913.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,7829.0,7740.0,7653.0,7566.0,7481.0,7396.0,7313.0,...,6028.0,5960.0,5892.0,5826.0,5760.0,5695.0,5631.0,5567.0,5504.0,5442.0
4,NaN,0.0,23658.0,-337.0,-332.0,23332.0,-659.0,-650.0,23018.0,-968.0,...,21397.0,-2566.0,-2530.0,21165.0,-2795.0,-2755.0,20943.0,-3014.0,-2971.0,20730.0


In [ ]:
diffbal.drop([0],axis=1,inplace=True)

To calculate the counts of positive and negative values for each row.

In [ ]:
counts_df=diffbal.apply(lambda row: pd.Series({'Positive': (row>0).sum(),
                                               'Negative': (row<0).sum()}),axis=1)
counts_df

,Positive,Negative
0,0,1
1,20,1
2,1,0
3,32,0
4,12,22
...,...,...
556975,7,1
556976,15,0
556977,33,0
556978,2,0


In [ ]:
merged_df=pd.concat([df,counts_df],axis=1)

In [ ]:
avgpayment=diffbal.mean(axis=1) #using difference means payment
avgpayment.head()

0   -25053.000000
1     1113.523810
2     5782.600000
3     6565.363636
4     6474.428571
dtype: float64

apply() is a method provided by the pandas library that is used to apply a function along an axis of a DataFrame (either rows or columns)

Function is lambda which is used to calculate the difference between the last valid value and the first valid value in each row of a DataFrame

In [ ]:
difference=df1.apply(lambda row: row[row.last_valid_index()]-row[row.first_valid_index()]
          if row.first_valid_index() is not None and row.last_valid_index() is not None
          else np.nan,axis=1) # finding the difference between first valid(NOT-NULL) transaction and last valid(NOT-NULL) transaction

In [ ]:
full_diff=difference
full_diff.head()

0    -25053.0
1     23384.0
2     28913.0
3    232582.0
4    226605.0
dtype: float64

If both indices are not None, it calculates the difference between the last valid value and the first valid value, otherwise it returns np.nan

##Reported Date History

In [ ]:
reporteddatehist=df['REPORTED DATE - HIST'].str.split(',',expand=True)
reporteddatehist.head(5)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,20180430,20180331,,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,20191231,20191130,20191031,20190930,20190831,20190731,20190630,20190531,20190430,20190331,...,None,None,None,None,None,None,None,None,None,None
2,20200131,20191231,20191130,20191031,20190930,20190831,,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,20170930,20170801,20170731,20170630,20170531,20170430,20170331,20170228,20170131,20161231,...,20150630,20150531,20150430,20150331,20150228,20150131,20141231,20141130,20141031,
4,20160229,20160131,20151231,20151130,20151031,20150930,20150831,20150731,20150630,20150531,...,20131130,20131031,20130930,20130831,20130731,20130630,20130531,20130430,20130331,


In [ ]:
from datetime import datetime
reporteddatehist=reporteddatehist.apply(lambda x: pd.to_datetime(x,format="%Y%m%d", errors='coerce'))

In [ ]:
reporteddatehist.head()

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,2018-04-30,2018-03-31,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,2019-12-31,2019-11-30,2019-10-31,2019-09-30,2019-08-31,2019-07-31,2019-06-30,2019-05-31,2019-04-30,2019-03-31,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,2020-01-31,2019-12-31,2019-11-30,2019-10-31,2019-09-30,2019-08-31,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,2017-09-30,2017-08-01,2017-07-31,2017-06-30,2017-05-31,2017-04-30,2017-03-31,2017-02-28,2017-01-31,2016-12-31,...,2015-06-30,2015-05-31,2015-04-30,2015-03-31,2015-02-28,2015-01-31,2014-12-31,2014-11-30,2014-10-31,NaT
4,2016-02-29,2016-01-31,2015-12-31,2015-11-30,2015-10-31,2015-09-30,2015-08-31,2015-07-31,2015-06-30,2015-05-31,...,2013-11-30,2013-10-31,2013-09-30,2013-08-31,2013-07-31,2013-06-30,2013-05-31,2013-04-30,2013-03-31,NaT


##Amount Overdue

In [ ]:
amtoverduehist=df['AMT OVERDUE - HIST'].str.split(',',expand=True)
amtoverduehist=amtoverduehist.apply(pd.to_numeric)
amtoverduehist.info()
amtoverduehist.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 556980 entries, 0 to 556979
Data columns (total 42 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       403150 non-null  float64
 1   1       337700 non-null  float64
 2   2       369223 non-null  float64
 3   3       355608 non-null  float64
 4   4       342168 non-null  float64
 5   5       332718 non-null  float64
 6   6       320463 non-null  float64
 7   7       306571 non-null  float64
 8   8       304074 non-null  float64
 9   9       297166 non-null  float64
 10  10      291050 non-null  float64
 11  11      285207 non-null  float64
 12  12      273345 non-null  float64
 13  13      256765 non-null  float64
 14  14      246761 non-null  float64
 15  15      241317 non-null  float64
 16  16      234000 non-null  float64
 17  17      226638 non-null  float64
 18  18      221210 non-null  float64
 19  19      215591 non-null  float64
 20  20      209987 non-null  float64
 21  21      19

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,37873.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,NaN,1014.0,1014.0,1014.0,1014.0,1014.0,1014.0,1014.0,983.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
amtoverduehist.drop(amtoverduehist.iloc[:,36:42],axis=1,inplace=True)
maxamtoverdue=amtoverduehist.max(axis=1)
maxamtoverdue.head(10)

0    37873.0
1     1452.0
2        0.0
3     1014.0
4        0.0
5        NaN
6        NaN
7     3064.0
8     1722.0
9        0.0
dtype: float64

In [ ]:
totalamtoverdue=amtoverduehist.sum(axis=1,numeric_only=True)
totalamtoverdue.head(10)
#When you use numeric_only=True with sum(axis=1), it indicates that only numeric (i.e., integer or float) columns will be considered when calculating the sum along each row.

0    37873.0
1     1452.0
2        0.0
3    16989.0
4        0.0
5        0.0
6        0.0
7    43545.0
8    33672.0
9        0.0
dtype: float64

In [ ]:
#No. of times the amount has been overdued
noverdue_ratio=(amtoverduehist.count(axis=1)-(amtoverduehist==0).sum(axis=1))/36
noverdue_ratio

0         0.027778
1         0.027778
2         0.000000
3         0.583333
4         0.000000
            ...   
556975    0.083333
556976    0.138889
556977    0.138889
556978    0.000000
556979    0.000000
Length: 556980, dtype: float64

## Amount Paid

Lot of empty data, We can ommit this

In [ ]:
amtpaidhist=df['AMT PAID - HIST'].str.split(',',expand=True)
amtpaidhist=amtpaidhist.apply(pd.to_numeric)
amtpaidhist.info()
amtpaidhist.head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 556980 entries, 0 to 556979
Data columns (total 38 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       95341 non-null  float64
 1   1       79779 non-null  float64
 2   2       74438 non-null  float64
 3   3       66957 non-null  float64
 4   4       64022 non-null  float64
 5   5       61767 non-null  float64
 6   6       54361 non-null  float64
 7   7       55660 non-null  float64
 8   8       54797 non-null  float64
 9   9       50791 non-null  float64
 10  10      51281 non-null  float64
 11  11      46921 non-null  float64
 12  12      41333 non-null  float64
 13  13      37808 non-null  float64
 14  14      33792 non-null  float64
 15  15      33911 non-null  float64
 16  16      34163 non-null  float64
 17  17      30592 non-null  float64
 18  18      30494 non-null  float64
 19  19      31077 non-null  float64
 20  20      29401 non-null  float64
 21  21      27724 non-null  float64
 

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
amtpaidhist.drop(amtpaidhist.iloc[:,36:38],inplace=True,axis=1)
totalamtpaid=amtpaidhist.sum(axis=1,numeric_only=True)
totalamtpaid.head(15)

0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
5           0.0
6           0.0
7           0.0
8           0.0
9           0.0
10          0.0
11          0.0
12     634325.0
13     671280.0
14    2688647.0
dtype: float64

In [ ]:
#Proportion of payments made in 36 months
npayments_ratio=(amtpaidhist.count(axis=1)-(amtpaidhist==0).sum(axis=1))/36
npayments_ratio.head(15)

#count(axis=1) measures the all non null values in a row

0     0.000000
1     0.000000
2     0.000000
3     0.000000
4     0.000000
5     0.000000
6     0.000000
7     0.000000
8     0.000000
9     0.000000
10    0.000000
11    0.000000
12    1.000000
13    0.916667
14    1.000000
dtype: float64

**Adding features in the database**

In [ ]:
df["total_amt"],df["full_difference"],df["overdue_times_ratio"],df["n_payments_ratio"],df["max_amt_overdue"],df["total_amt_overdue"] = totalamtpaid,full_diff,noverdue_ratio,npayments_ratio,maxamtoverdue,totalamtoverdue


##Write off Amount

In [ ]:
df['WRITE-OFF-AMT']

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
556975    0.0
556976    0.0
556977    0.0
556978    0.0
556979    0.0
Name: WRITE-OFF-AMT, Length: 556980, dtype: float64

In [ ]:
df['WRITE-OFF-AMT'] .nunique()

1009

In [ ]:
#Later
Total_write_off_df=df.groupby('ID')['WRITE-OFF-AMT'].sum().to_frame()  #Total Write-Off amount for a given ID
Total_write_off_df

,WRITE-OFF-AMT
ID,
1,0.0
2,0.0
3,0.0
7,0.0
8,0.0
...,...
143390,0.0
143391,0.0
143393,0.0


In [ ]:
Total_write_off_df.value_counts()

WRITE-OFF-AMT
0.0              127694
352886.0              9
143614.0              4
238.0                 3
399717.0              2
                  ...  
27152.0               1
27361.0               1
27784.0               1
27853.0               1
8797356.0             1
Length: 918, dtype: int64

The amount of 917 customers have been written off by the bank

##Account Status

This field displays the status of loan account

In [ ]:
df['ACCOUNT-STATUS'].unique()

array(['Delinquent', 'Active', 'Closed', 'SUIT FILED (WILFUL DEFAULT)',
       'Written Off', 'Suit Filed', 'Restructured', 'Settled',
       'WILFUL DEFAULT', 'Cancelled', 'Sold/Purchased'], dtype=object)

Being delinquent refers to the state of being past due on a debt. Delinquency occurs as soon as a borrower misses a payment on a loan, which can affect their credit score.

Good Account: 'Active', 'Closed','Restructured', 'Settled'

Bad Account: 'Delinquent', 'SUIT FILED (WILFUL DEFAULT)', 'Written Off', 'Suit Filed', 'WILFUL DEFAULT', 'Cancelled', 'Sold/Purchased'

In [ ]:
df['ACCOUNT-STATUS'].value_counts()

Closed                         319447
Active                         198897
Delinquent                      32410
Written Off                      2931
Suit Filed                       2062
Settled                           624
Restructured                      510
SUIT FILED (WILFUL DEFAULT)        70
WILFUL DEFAULT                     27
Cancelled                           1
Sold/Purchased                      1
Name: ACCOUNT-STATUS, dtype: int64

In [ ]:
df["ACCOUNT-STATUS"]=df["ACCOUNT-STATUS"].replace(['Delinquent','SUIT FILED (WILFUL DEFAULT)','Written Off','Suit Filed','Restructured','SUIT FILED (WILFUL DEFAULT)','WILFUL DEFAULT','Cancelled','Sold/Purchased'],'Bad')

In [ ]:
categories = ['Bad', 'Active', 'Closed']
df['Encoded_Account_Status'] = df['ACCOUNT-STATUS'].astype('category').cat.set_categories(categories).cat.codes
df['Encoded_Account_Status'] = df['Encoded_Account_Status'].replace(-1, float('nan'))
df.drop("ACCOUNT-STATUS",axis = 1,inplace = True)

df['ACCOUNT-STATUS'].astype('category'): Converts the 'ACCOUNT-STATUS' column to a categorical data type.

.cat.set_categories(categories): Sets the categories of the categorical data type to the specified categories list.

.cat.codes: Encodes the categories as numerical codes.

##Asset Class

In [ ]:
df['ASSET_CLASS'].value_counts()

Standard                   245736
SubStandard                  5293
Doubtful                     2575
Special Mention Account      2497
Loss                          872
1                             362
01                            176
2                               3
Name: ASSET_CLASS, dtype: int64

In [ ]:
df["ASSET_CLASS"]=df['ASSET_CLASS'].replace(['Doubtful','Special Mention Account','Loss','1','01','2'],'Bad')

In [ ]:
df['ASSET_CLASS'].value_counts()

Standard       245736
Bad              6485
SubStandard      5293
Name: ASSET_CLASS, dtype: int64

In [ ]:
categories = ['Bad', 'SubStandard', 'Standard']
df['Encoded_Asset_Class'] = df['ASSET_CLASS'].astype('category').cat.set_categories(categories).cat.codes
df['Encoded_Asset_Class'] = df['Encoded_Asset_Class'].replace(-1, float('nan'))

df.drop("ASSET_CLASS",axis = 1,inplace = True)

##Ownership ID

In [ ]:
df["OWNERSHIP-IND"].value_counts()

Individual          480924
Joint                36850
Guarantor            34521
Primary               4547
Supl Card Holder       138
Name: OWNERSHIP-IND, dtype: int64

In [ ]:
df["OWNERSHIP-IND"]=df["OWNERSHIP-IND"].replace(['Joint','Guarantor','Primary','Supl Card Holder'],'Other')

In [ ]:
df['OWNERSHIP-IND'].value_counts()

Individual    480924
Other          76056
Name: OWNERSHIP-IND, dtype: int64

In [ ]:
categories = ['Other','Individual']
df['Encoded_OWNERSHIP-IND'] = df['OWNERSHIP-IND'].astype('category').cat.set_categories(categories).cat.codes
df['Encoded_OWNERSHIP-IND'] = df['Encoded_OWNERSHIP-IND'].replace(-1, 0)
## We do this -1 to 0 because this code will give default as -1 to Other and we want it to be 0 as in previous case there
## were Nan values and this code gives Nan -1 so for uniformity in our dataset we did this
df.drop("OWNERSHIP-IND",axis = 1,inplace = True)

In [ ]:
df['Encoded_OWNERSHIP-IND'].value_counts()

1    480924
0     76056
Name: Encoded_OWNERSHIP-IND, dtype: int64

## Installment Frequency

In [ ]:
df['INSTALLMENT-FREQUENCY']=df['INSTALLMENT-FREQUENCY'].replace(['F01','F02','F03','F04','F05','F06','F07'], [52,26,12,6,4,2,1])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 556980 entries, 0 to 556979
Data columns (total 31 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ID                         556980 non-null  int64  
 1   SELF-INDICATOR             556980 non-null  bool   
 2   MATCH-TYPE                 556980 non-null  object 
 3   ACCT-TYPE                  556980 non-null  object 
 4   CONTRIBUTOR-TYPE           556980 non-null  object 
 5   DATE-REPORTED              553303 non-null  object 
 6   DISBURSED-DT               525096 non-null  object 
 7   CLOSE-DT                   308331 non-null  object 
 8   LAST-PAYMENT-DATE          238736 non-null  object 
 9   CREDIT-LIMIT/SANC AMT      15103 non-null   object 
 10  DISBURSED-AMT/HIGH CREDIT  556980 non-null  object 
 11  INSTALLMENT-AMT            138103 non-null  object 
 12  CURRENT-BAL                556748 non-null  object 
 13  INSTALLMENT-FREQUENCY      13

##Installment Amount

In [ ]:
df['INSTALLMENT-AMT'].head()

0              NaN
1    1,405/Monthly
2              NaN
3              NaN
4              NaN
Name: INSTALLMENT-AMT, dtype: object

In [ ]:
installamt=df['INSTALLMENT-AMT'].str.split('/',expand=True)
installamt.head()  

,0,1,2
0,NaN,NaN,NaN
1,"1,405",Monthly,None
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN


In [ ]:
df['INSTALLMENT-AMT'] = installamt.iloc[:,0]
df['INSTALLMENT-AMT'].head(15)

0        NaN
1      1,405
2        NaN
3        NaN
4        NaN
5        NaN
6      7,934
7        NaN
8        NaN
9        NaN
10       NaN
11    31,889
12    17,620
13       NaN
14    74,684
Name: INSTALLMENT-AMT, dtype: object

In [ ]:
df['INSTALLMENT-AMT']=df['INSTALLMENT-AMT'].astype(str).str.replace(',','')   #str.replace("","") is a function
df['INSTALLMENT-AMT']=df['INSTALLMENT-AMT'].astype(float)
df['INSTALLMENT-AMT'].head(5)

0       NaN
1    1405.0
2       NaN
3       NaN
4       NaN
Name: INSTALLMENT-AMT, dtype: float64

##Disbursed Amount

In [ ]:
df["DISBURSED-AMT/HIGH CREDIT"].head()

0      37,352
1      44,000
2    1,45,000
3    3,00,000
4    2,75,000
Name: DISBURSED-AMT/HIGH CREDIT, dtype: object

In [ ]:
df["DISBURSED-AMT/HIGH CREDIT"]=df["DISBURSED-AMT/HIGH CREDIT"].astype(str).str.replace(',','')
df["DISBURSED-AMT/HIGH CREDIT"]=df["DISBURSED-AMT/HIGH CREDIT"].astype(float)
df["DISBURSED-AMT/HIGH CREDIT"].head()

0     37352.0
1     44000.0
2    145000.0
3    300000.0
4    275000.0
Name: DISBURSED-AMT/HIGH CREDIT, dtype: float64

##Close Date

In [ ]:
df['CLOSE-DT'].head()

0                    NaN
1                    NaN
2                    NaN
3    2017-09-21 00:00:00
4    2016-02-01 00:00:00
Name: CLOSE-DT, dtype: object

In [ ]:
df['CLOSE-DT']=pd.to_datetime(df['CLOSE-DT'],errors='coerce')  # errors='coerce' it means that if there is an error encountered during the operation, the problematic values will be replaced with a special value (typically NaN or NaT for numeric and datetime data, respectively)
df["CLOSE-DT"].head()

0          NaT
1          NaT
2          NaT
3   2017-09-21
4   2016-02-01
Name: CLOSE-DT, dtype: datetime64[ns]

## Last Payment Date, Disbursed Date and Close Date

doubt in this

In [ ]:
df['CLOSE-DT'] = pd.to_datetime(df['CLOSE-DT'], errors='coerce')
df['LAST-PAYMENT-DATE'] = pd.to_datetime(df['LAST-PAYMENT-DATE'])
df['DISBURSED-DT'] = pd.to_datetime(df['DISBURSED-DT'])

In [ ]:
df['DISBURSED-DT'][316]

Timestamp('2015-07-04 00:00:00')

In [ ]:
df['LAST-PAYMENT-DATE'][316]

Timestamp('2015-07-03 00:00:00')

In [ ]:
df['LAST-PAYMENT-DATE'] = pd.to_datetime(df['LAST-PAYMENT-DATE'])
df['DISBURSED-DT'] = pd.to_datetime(df['DISBURSED-DT'])

def compute_npdays(row):
    if pd.isna(row["LAST-PAYMENT-DATE"]):
        return (row['CLOSE-DT'] - row['DISBURSED-DT']).days
    else:
        return (row['LAST-PAYMENT-DATE'] - row['DISBURSED-DT']).days

df['npdays'] = df.apply(compute_npdays, axis=1)

In [ ]:
df[df['npdays']<0]

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,...,total_amt,full_difference,overdue_times_ratio,n_payments_ratio,max_amt_overdue,total_amt_overdue,Encoded_Account_Status,Encoded_Asset_Class,Encoded_OWNERSHIP-IND,npdays
316,52,False,PRIMARY,Commercial Vehicle Loan,PRB,2015-07-31 00:00:00,2015-07-04,NaT,2015-07-03,NaN,...,0.0,NaN,0.000000,0.000000,NaN,0.0,1.0,NaN,0,-1.0
994,144,False,PRIMARY,Commercial Vehicle Loan,PRB,2016-05-31 00:00:00,2016-05-25,NaT,2016-05-21,NaN,...,0.0,NaN,0.000000,0.000000,NaN,0.0,1.0,NaN,1,-4.0
1567,266,False,PRIMARY,Commercial Vehicle Loan,PRB,2016-05-31 00:00:00,2016-05-25,NaT,2016-05-21,NaN,...,0.0,NaN,0.000000,0.000000,NaN,0.0,1.0,NaN,1,-4.0
1638,273,False,PRIMARY,Construction Equipment Loan,PRB,2019-04-30 00:00:00,2017-05-02,2019-04-08,2017-04-29,NaN,...,430440.0,797418.0,0.055556,0.305556,29074.0,31231.0,2.0,NaN,0,-3.0
1672,273,False,PRIMARY,Construction Equipment Loan,PRB,2019-04-30 00:00:00,2017-05-16,2019-04-30,2017-04-29,NaN,...,246800.0,449400.0,0.027778,0.305556,8739.0,8739.0,2.0,NaN,0,-17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554254,142534,False,PRIMARY,Gold Loan,COP,2019-08-31 00:00:00,2018-10-01,NaT,2018-06-30,NaN,...,0.0,-563.0,0.000000,0.000000,0.0,0.0,1.0,2.0,1,-93.0
554554,142640,False,PRIMARY,Other,COP,2019-08-31 00:00:00,2019-08-09,NaT,2019-07-28,NaN,...,0.0,0.0,0.000000,0.000000,0.0,0.0,1.0,2.0,1,-12.0
554559,142640,False,PRIMARY,Other,COP,2019-08-31 00:00:00,2019-08-09,NaT,2019-07-28,NaN,...,0.0,0.0,0.000000,0.000000,0.0,0.0,1.0,2.0,1,-12.0
556416,143159,False,PRIMARY,Credit Card,COP,2019-08-31 00:00:00,2018-08-28,NaT,2018-01-14,NaN,...,0.0,0.0,0.000000,0.000000,0.0,0.0,1.0,2.0,1,-226.0


In [ ]:
##From the above data frame we can see that the certain days are comping negative which is not possible in real life
##So we will replace them with Nan

df.loc[df['npdays'] < 0, 'npdays'] = np.nan

In [ ]:
df[df['npdays']==0]

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,...,total_amt,full_difference,overdue_times_ratio,n_payments_ratio,max_amt_overdue,total_amt_overdue,Encoded_Account_Status,Encoded_Asset_Class,Encoded_OWNERSHIP-IND,npdays
670,94,False,PRIMARY,Commercial Vehicle Loan,NBF,2019-12-31 00:00:00,2019-09-17,NaT,2019-09-17,NaN,...,0.0,39658.0,0.000000,0.000000,0.0,0.0,1.0,2.0,1,0.0
706,95,False,PRIMARY,Commercial Vehicle Loan,NBF,2019-12-31 00:00:00,2019-09-17,NaT,2019-09-17,NaN,...,0.0,39658.0,0.000000,0.000000,0.0,0.0,1.0,2.0,1,0.0
1896,315,False,PRIMARY,Consumer Loan,NBF,2019-09-30 00:00:00,2019-08-22,2019-08-22,2019-08-22,NaN,...,0.0,0.0,0.000000,0.000000,0.0,0.0,2.0,NaN,1,0.0
2248,375,False,PRIMARY,Business Loan Priority Sector Agriculture,NAB,2017-01-20 00:00:00,2016-06-18,2017-01-20,2016-06-18,NaN,...,0.0,NaN,0.000000,0.000000,NaN,0.0,2.0,NaN,1,0.0
3097,502,False,PRIMARY,Commercial Vehicle Loan,NBF,2019-12-31 00:00:00,2015-10-01,2018-01-30,2015-10-01,NaN,...,0.0,2501902.0,0.305556,0.000000,421016.0,2768405.0,2.0,2.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555973,142980,False,PRIMARY,Gold Loan,NAB,2018-05-19 00:00:00,2017-05-22,2018-05-19,2017-05-22,NaN,...,0.0,82000.0,0.000000,0.000000,0.0,0.0,2.0,2.0,1,0.0
555978,142980,False,PRIMARY,Gold Loan,NAB,2020-01-31 00:00:00,2019-05-16,NaT,2019-05-16,NaN,...,0.0,0.0,0.000000,0.000000,0.0,0.0,1.0,2.0,1,0.0
555982,142981,False,PRIMARY,Gold Loan,NBF,2019-12-31 00:00:00,2019-11-30,2019-12-20,2019-11-30,NaN,...,0.0,0.0,0.000000,0.000000,0.0,0.0,2.0,2.0,1,0.0
555984,142981,False,PRIMARY,Tractor Loan,NBF,2020-01-31 00:00:00,2019-11-13,NaT,2019-11-13,NaN,...,660000.0,0.0,0.000000,0.083333,0.0,0.0,1.0,NaN,1,0.0


Below are the series of the group by functions which we have used to make new feature for our ltfs data from the customers past history .

We have implied several central tendancies which are best suitable for the give situation like when the data is categorical we used mode , when there were extreme values then we used median and in several cases we have used mean of all the past history of the given customer.

We have also calculated the ratio of the no. of things that happened to the total no. of things that could have happened , this helped us in getting some picture of the behaviour of the customer.

In [ ]:
tenure=df.groupby('ID')['TENURE'].median()
assetclass=df.groupby(['ID'])['Encoded_Asset_Class'].agg(pd.Series.mode)
total_disbursed_amount=df.groupby('ID')['DISBURSED-AMT/HIGH CREDIT'].sum()
account_status=df.groupby(['ID'])['Encoded_Account_Status'].agg(pd.Series.mode)
total_writeoffamt = df.groupby(['ID'])['WRITE-OFF-AMT'].sum()

In [ ]:
total_amt = df.groupby(["ID"])["total_amt"].sum()
last_first_current_balance_diff = df.groupby(["ID"])["full_difference"].mean()
times_amount_overdue_ratio =df.groupby(["ID"])["overdue_times_ratio"].mean()
avg_days_loan =df.groupby(["ID"])["npdays"].mean()
times_paymentdone_ratio = df.groupby(["ID"])["n_payments_ratio"].mean()
installment_frequency = df.groupby(['ID'])['INSTALLMENT-FREQUENCY'].agg(pd.Series.mode)
installment_amount = df.groupby(['ID'])['INSTALLMENT-AMT'].median()
total_positive =  merged_df.groupby(["ID"])["Positive"].sum()
total_negative = merged_df.groupby(["ID"])["Negative"].sum()
self_indicator_ratio = df.groupby(["ID"])['SELF-INDICATOR'].mean()
max_overdue = df.groupby(["ID"])["max_amt_overdue"].max()
avg_overdue = df.groupby(["ID"])["total_amt_overdue"].mean()
ownership = df.groupby(["ID"])['Encoded_OWNERSHIP-IND'].agg(pd.Series.mode)

<ipython-input-83-04a6608cfaf7>:6: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  installment_frequency = df.groupby(['ID'])['INSTALLMENT-FREQUENCY'].agg(pd.Series.mode)
<ipython-input-83-04a6608cfaf7>:6: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'int'
  installment_frequency = df.groupby(['ID'])['INSTALLMENT-FREQUENCY'].agg(pd.Series.mode)
<ipython-input-83-04a6608cfaf7>:6: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'int'
  installment_frequency = df.groupby(['ID'])['INSTALLMENT-FREQUENCY'].agg(pd.Series.mode)
<ipython-input-83-04a6608cfaf7>:6: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'int'
  installment_frequency = df.groupby(['ID'])['INSTALLMENT-FREQUENCY'].agg(pd.Series.mode)
<ipython-input-83-04a6608cfaf7>:6: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'int'
  installment_f

In [ ]:
final_df = pd.DataFrame({'P_tenure': tenure, 'P_assetclass': assetclass, 'P_total_disbursed_amount': total_disbursed_amount,'P_account_status':account_status,'P_total_writeoffamt':total_writeoffamt,"P_totalamtpaid":total_amt,"P_avg_current_bal_difference":last_first_current_balance_diff,"P_overdue_time_ratio":times_amount_overdue_ratio,"P_total_positive":total_positive,"P_total_negative":total_negative,"self_indicator_ratio":self_indicator_ratio,'P_paymentdone_ratio':times_paymentdone_ratio,'P_avg_days_loan':avg_days_loan,'P_installment_frequency':installment_frequency,'P_installment_amount':installment_amount,'P_max_overdue':max_overdue,'P_avg_overdue':avg_overdue,'P_ownership':ownership})
final_df.head(5)

,P_tenure,P_assetclass,P_total_disbursed_amount,P_account_status,P_total_writeoffamt,P_totalamtpaid,P_avg_current_bal_difference,P_overdue_time_ratio,P_total_positive,P_total_negative,self_indicator_ratio,P_paymentdone_ratio,P_avg_days_loan,P_installment_frequency,P_installment_amount,P_max_overdue,P_avg_overdue,P_ownership
ID,,,,,,,,,,,,,,,,,,
1,60.0,2.0,2201352.0,2.0,0.0,0.00,124579.666667,0.231481,135,26,0.555556,0.000000,1006.875000,12,4669.5,37873.0,14836.777778,1
2,48.0,2.0,18117081.0,1.0,0.0,22432902.00,440017.333333,0.066239,196,0,0.076923,0.326923,747.615385,12,31889.0,11423.0,4819.538462,1
3,66.0,2.0,3708369.0,2.0,0.0,1328572.98,55994.571429,0.043011,140,41,0.064516,0.066308,568.909091,12,17608.5,77626.0,44104.387097,1
7,16.5,[],1272553.0,"[1.0, 2.0]",0.0,0.00,200216.750000,0.000000,55,2,0.500000,0.000000,759.000000,12,5571.0,0.0,0.000000,1
8,51.0,2.0,2025216.0,1.0,0.0,202930.00,190484.714286,0.067460,74,40,0.142857,0.051587,1036.750000,12,2301.0,47857.0,43521.000000,0


In [ ]:
final_df.info()
len(final_df)
final_df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128655 entries, 1 to 143395
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   P_tenure                      54347 non-null   float64
 1   P_assetclass                  128655 non-null  object 
 2   P_total_disbursed_amount      128655 non-null  float64
 3   P_account_status              128655 non-null  object 
 4   P_total_writeoffamt           128655 non-null  float64
 5   P_totalamtpaid                128655 non-null  float64
 6   P_avg_current_bal_difference  128634 non-null  float64
 7   P_overdue_time_ratio          128655 non-null  float64
 8   P_total_positive              128655 non-null  int64  
 9   P_total_negative              128655 non-null  int64  
 10  self_indicator_ratio          128655 non-null  float64
 11  P_paymentdone_ratio           128655 non-null  float64
 12  P_avg_days_loan               102780 non-nul

,P_tenure,P_assetclass,P_total_disbursed_amount,P_account_status,P_total_writeoffamt,P_totalamtpaid,P_avg_current_bal_difference,P_overdue_time_ratio,P_total_positive,P_total_negative,self_indicator_ratio,P_paymentdone_ratio,P_avg_days_loan,P_installment_frequency,P_installment_amount,P_max_overdue,P_avg_overdue,P_ownership
ID,,,,,,,,,,,,,,,,,,
1,60.0,2.0,2201352.0,2.0,0.0,0.00,124579.666667,0.231481,135,26,0.555556,0.000000,1006.875000,12,4669.5,37873.0,14836.777778,1
2,48.0,2.0,18117081.0,1.0,0.0,22432902.00,440017.333333,0.066239,196,0,0.076923,0.326923,747.615385,12,31889.0,11423.0,4819.538462,1
3,66.0,2.0,3708369.0,2.0,0.0,1328572.98,55994.571429,0.043011,140,41,0.064516,0.066308,568.909091,12,17608.5,77626.0,44104.387097,1
7,16.5,[],1272553.0,"[1.0, 2.0]",0.0,0.00,200216.750000,0.000000,55,2,0.500000,0.000000,759.000000,12,5571.0,0.0,0.000000,1
8,51.0,2.0,2025216.0,1.0,0.0,202930.00,190484.714286,0.067460,74,40,0.142857,0.051587,1036.750000,12,2301.0,47857.0,43521.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143390,121.0,[],2087601.0,1.0,0.0,0.00,109985.500000,0.000000,6,0,0.500000,0.000000,100.000000,[],NaN,0.0,0.000000,1
143391,NaN,[],275630.0,1.0,0.0,0.00,201740.000000,0.000000,3,0,1.000000,0.000000,NaN,[],NaN,0.0,0.000000,1
143393,24.0,2.0,704552.0,1.0,0.0,0.00,234698.000000,0.090278,67,1,0.250000,0.000000,1144.000000,[],NaN,22920.0,33621.500000,1


In [ ]:
final_df = final_df.reset_index()

In [ ]:
final_df.head()

,ID,P_tenure,P_assetclass,P_total_disbursed_amount,P_account_status,P_total_writeoffamt,P_totalamtpaid,P_avg_current_bal_difference,P_overdue_time_ratio,P_total_positive,P_total_negative,self_indicator_ratio,P_paymentdone_ratio,P_avg_days_loan,P_installment_frequency,P_installment_amount,P_max_overdue,P_avg_overdue,P_ownership
0,1,60.0,2.0,2201352.0,2.0,0.0,0.00,124579.666667,0.231481,135,26,0.555556,0.000000,1006.875000,12,4669.5,37873.0,14836.777778,1
1,2,48.0,2.0,18117081.0,1.0,0.0,22432902.00,440017.333333,0.066239,196,0,0.076923,0.326923,747.615385,12,31889.0,11423.0,4819.538462,1
2,3,66.0,2.0,3708369.0,2.0,0.0,1328572.98,55994.571429,0.043011,140,41,0.064516,0.066308,568.909091,12,17608.5,77626.0,44104.387097,1
3,7,16.5,[],1272553.0,"[1.0, 2.0]",0.0,0.00,200216.750000,0.000000,55,2,0.500000,0.000000,759.000000,12,5571.0,0.0,0.000000,1
4,8,51.0,2.0,2025216.0,1.0,0.0,202930.00,190484.714286,0.067460,74,40,0.142857,0.051587,1036.750000,12,2301.0,47857.0,43521.000000,0


In [ ]:
from google.colab import files

csv_name='final_df.csv'
final_df.to_csv(csv_name,index=False)
files.download(csv_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>